# tornado获取post数据

### 一次性获取post所有数据

In [7]:
import requests
import json
from pprint import pprint
url = 'http://127.0.0.1:8760/api/portal'
payload = {"organization_id":"1a84f6714f524c65b3e82fc4400276c4",
        "ent_code":"paypals",
        "product_code":"sms",
        "amount":"1",
        "type":"renew",
        "ent_name":"XXXXX",
        "account_id":57,
        "account_name":"admin@paypals.com",
        "product_specification_info":{
            "fee_type":"count",
            "specification_id":37,
            "specification_no":"sms-standard"}
       }

res = requests.post(url, data=payload, timeout=5)
# res = requests.post(url, json=payload, timeout=5)
pprint(res.json())

{'errMsg': '', 'remain_amount': 0, 'status': 0}


注意: requests在post数据时,
当以data作为关键字参数时,传递的是字符串形式的bytes,
当以json作为关键字参数时,传递的是json格式的bytes.

In [ ]:
import json
from tornado import web

class PortalNoticeHandler(web.RequestHandler):

    def post(self):
        # 获取body中的所有post数据,注意这个post的数据必须是以json格式传递的,否则json.loads会失败!
        data = json.loads(self.request.body.decode())
        self.write({'status': 0, 'errMsg': '', 'remain_amount': 0})
        self.finish()

    def get(self):
        pass

In [ ]:
from tornado import web

class PortalNoticeHandler(web.RequestHandler):

    def post(self):
        try:
            data = {}
            for key in self.request.arguments:
                data[key] = self.get_argument(key)
            pprint(data)
            self.write({'status': 0, 'errMsg': '', 'remain_amount': 0})
        except Exception as e:
            logging.error('Get portal data error: {}'.format(e))
            self.write({'status': 1, 'errMsg': 'data formatter error'})

        self.finish()

    def get(self):
        pass

{'account_id': '57',
 'account_name': 'admin@paypals.com',
 'amount': '1',
 'ent_code': 'paypals',
 'ent_name': 'XXXXX',
 'organization_id': '1a84f6714f524c65b3e82fc4400276c4',
 'product_code': 'sms',
 'product_specification_info': 'specification_no',
 'type': 'renew'}

tornado的get和post提交的参数都可以通过self.get_argument()获得

### 判断参数是否存在并获取

In [ ]:
if self.request.arguments.has_key("username"):
    user = self.get_argument('username', 'admin')

### tornado获取参数的方式

In [ ]:
1. def get_argument(self, name, default=_ARG_DEFAULT, strip=True)

2. def get_arguments(self, name, strip=True)
    
3. def get_body_argument(self, name, default=_ARG_DEFAULT, strip=True)
    
4. def get_body_arguments(self, name, strip=True)
    
5. def get_query_argument(self, name, default=_ARG_DEFAULT, strip=True)
    
6. def get_query_arguments(self, name, strip=True)
    
7. self.request.body 

POST数据 通常指 POST 时 body 中的数据。而 QueryString （URL）中也有可以带参数（通常是 GET 时的参数）。

如果 POST 时同时存在 QueryString 和 body data，那么 get_argument 获取得到的其实是 QueryString 中的数据，而不是 body 中的数据。

get_body_argument 这个方法是获取 body 中的参数值。